# STEP 1b: Extracting and Initial Pre-Processing (b)

### I have used RethinkDB to store my initial scraped data. 

In [1]:
#Files would be saved in db file
from rethinkdb import RethinkDB
r = RethinkDB()


In [2]:
#Connection to RethinkDB server
r.connect( "localhost", 28015).repl()


###### I have cleaned my scraped data using basic string manipulations and one function from nltk corpus namely  word tokenize.

In [3]:
#Data Cleaning

import re, string, unicodedata
import nltk
import contractions
import inflect
from nltk import word_tokenize, sent_tokenize

In [4]:
#Database and table creation
r.db("Reddit").table_create("SubReddit").run()


{'config_changes': [{'new_val': {'db': 'Reddit',
    'durability': 'hard',
    'id': '9ebcb9e8-84e7-4843-bb4a-e0c185bec364',
    'indexes': [],
    'name': 'SubReddit',
    'primary_key': 'id',
    'shards': [{'nonvoting_replicas': [],
      'primary_replica': 'aditya_acer_aspire_w79',
      'replicas': ['aditya_acer_aspire_w79']}],
    'write_acks': 'majority',
    'write_hook': None},
   'old_val': None}],
 'tables_created': 1}

In [5]:
import praw

# Account details to use the praw API

client_id = "5bhwoommVfsvdw"
secret = "ChLYwsJ7x7qW6KVUxPycPKx6Er4"
app_name = "Flair Detector"
username = "flairdetector"
pw = "flairdetector" 

In [6]:
#Cr#eating a Reddit Object
reddit_obj = praw.Reddit(client_id=client_id,client_secret=secret,user_agent=app_name,username=username,password=pw)

#Getting values of subreddit r/india from the reddit object

subred_india = reddit_obj.subreddit('india')

###### All the flairs were selected on two parameters:
1. I picked all the flairs that appeared on the r/india HomePage. (Manually Scraped)
2. Secondly I looked at the frequency of flairs of almost top 10000 posts sorted on the basis of "Top", "Hot",        "Controversial" using the praw API (Eg: AMA, Unverified     )
3. Flairs that were related to Recent or Ongoing Events. (Eg: Coronavirus, CAA-NRC)

In [1]:
# Following is the list of flairs selected

flairs=["[R]eddiquette","Politics","Non-Political","Business/Finance","Unverified","Science/Technology","Sports","Photography","CAA-NRC","Food","AskIndia","Policy/Economy","Scheduled","AMA","Coronavirus"]


In [2]:
print(len(flairs))

15


## To clean the data received from praw API for each post, I followed a number of steps:

1. Removed symbolf from the text like {[(!@#$|%^.;:?><*=`~\-/_,&+)]}'"
2. Remove all '\n' Character(s)
3. I also expand words in the string such as *didn't* to did not which would help the models accuracy to be trained    better.
4. Now I tokenize the words i.e. break up the string into words.


In [9]:
def clean(t):    
    t_old = str(t)
    t_old = t_old.translate({ord(i): None for i in '{[(!@#$|%^.;:?><*=`~\-/_,&+)]}'})
    t_old = t_old.replace('\n','')
    t_old = t_old.replace('"','')
    t_old = t_old.replace("'",'')
    t_old = contractions.fix(t_old)
    t_new = nltk.word_tokenize(t_old)
    word_list=[]
    for word in t_new:
        word1=word.lower()
        word_list.append(word1)
    p = inflect.engine()
    new_words = []
    for word in word_list:
        if word.isdigit():
            new_word = p.number_to_words(word)
            new_words.append(new_word)
        else:
            new_words.append(word)
    if len(new_words) == 0:
        return None
    else:
        return new_words

##### For searching the posts using PRAW API, flair-wise I had to create a string like query such as : 
    "flair:{Name of the Flair}" // Only required for Coronavirus as it did not work with the below query.
    "flair_name:{Name of the Flair}"

###### I have picked 100 posts for each flair using the Reddit praw API and selected the following criterion for each posr
1. Title: Any Reddit post has to have a title necessarily.
2. URL: The url of a Reddit Post is not always the link to the post but many a times to Articles that are relevant    to the post and its contents.
3. The Body of a Reddit Post is also saved
4. Number of Comments: This field was saved specifically with the EDA in mind.
5. Comments: I picked out the top 20 comments for each post (removing all type of MoreComment Objects to increase      speed of Data Extraction)
Note: Reddit Comment have MoreComment Objects with are effectively Threads of long Comments

In [10]:
# Values for Flair: Coronavirus had to be entered separately, because its search query followed a slightly different approach.
cnt=0
post={}
req_posts=subred_india.search(query=str("flair:"+'"'+"Coronavirus"+'"'),limit=100)
for item in req_posts:
    cnt=cnt+1
    print("Entry No "+str(cnt))
    post = {
    "title":clean(item.title),
    "url":str(item.url),
    "body":clean(item.selftext),
    "no_comm":item.num_comments,
    "flair":str("Coronavirus"),
    }
    item.comments.replace_more(limit=0)
    comment = ''
    count=0
    for top_level_comment in item.comments:
        comment = comment + ' ' + top_level_comment.body
        count=count+1     
        if(count > 20):
            break
    post["comment"] = clean(comment)
    r.db("Reddit").table("SubReddit").insert(post).run()
print("Done")

Entry No 1
Entry No 2
Entry No 3
Entry No 4
Entry No 5
Entry No 6
Entry No 7
Entry No 8
Entry No 9
Entry No 10
Entry No 11
Entry No 12
Entry No 13
Entry No 14
Entry No 15
Entry No 16
Entry No 17
Entry No 18
Entry No 19
Entry No 20
Entry No 21
Entry No 22
Entry No 23
Entry No 24
Entry No 25
Entry No 26
Entry No 27
Entry No 28
Entry No 29
Entry No 30
Entry No 31
Entry No 32
Entry No 33
Entry No 34
Entry No 35
Entry No 36
Entry No 37
Entry No 38
Entry No 39
Entry No 40
Entry No 41
Entry No 42
Entry No 43
Entry No 44
Entry No 45
Entry No 46
Entry No 47
Entry No 48
Entry No 49
Entry No 50
Entry No 51
Entry No 52
Entry No 53
Entry No 54
Entry No 55
Entry No 56
Entry No 57
Entry No 58
Entry No 59
Entry No 60
Entry No 61
Entry No 62
Entry No 63
Entry No 64
Entry No 65
Entry No 66
Entry No 67
Entry No 68
Entry No 69
Entry No 70
Entry No 71
Entry No 72
Entry No 73
Entry No 74
Entry No 75
Entry No 76
Entry No 77
Entry No 78
Entry No 79
Entry No 80
Entry No 81
Entry No 82
Entry No 83
Entry No 84
E

In [11]:
#Values for the remaining flairs entered through this snippet.
post={}
cnt=0
for ele in flairs:    
    req_posts=subred_india.search(query=str("flair_name:"+'"'+ele+'"'),limit=100)
    for item in req_posts:
        cnt=cnt+1
        print("Entry No "+str(cnt))
        post = {
        "title":clean(item.title),
        "url":str(item.url),
        "body":clean(item.selftext),
        "no_comm":item.num_comments,
        "flair":str(ele),
        }
        item.comments.replace_more(limit=0)
        comment = ''
        count=0
        for top_level_comment in item.comments:
            comment = comment + ' ' + top_level_comment.body
            count = count+1     
            if(count > 20):
                break
        post["comment"] = clean(comment)
        r.db("Reddit").table("SubReddit").insert(post).run()
print("Done")


Entry No 1
Entry No 2
Entry No 3
Entry No 4
Entry No 5
Entry No 6
Entry No 7
Entry No 8
Entry No 9
Entry No 10
Entry No 11
Entry No 12
Entry No 13
Entry No 14
Entry No 15
Entry No 16
Entry No 17
Entry No 18
Entry No 19
Entry No 20
Entry No 21
Entry No 22
Entry No 23
Entry No 24
Entry No 25
Entry No 26
Entry No 27
Entry No 28
Entry No 29
Entry No 30
Entry No 31
Entry No 32
Entry No 33
Entry No 34
Entry No 35
Entry No 36
Entry No 37
Entry No 38
Entry No 39
Entry No 40
Entry No 41
Entry No 42
Entry No 43
Entry No 44
Entry No 45
Entry No 46
Entry No 47
Entry No 48
Entry No 49
Entry No 50
Entry No 51
Entry No 52
Entry No 53
Entry No 54
Entry No 55
Entry No 56
Entry No 57
Entry No 58
Entry No 59
Entry No 60
Entry No 61
Entry No 62
Entry No 63
Entry No 64
Entry No 65
Entry No 66
Entry No 67
Entry No 68
Entry No 69
Entry No 70
Entry No 71
Entry No 72
Entry No 73
Entry No 74
Entry No 75
Entry No 76
Entry No 77
Entry No 78
Entry No 79
Entry No 80
Entry No 81
Entry No 82
Entry No 83
Entry No 84
E

Entry No 640
Entry No 641
Entry No 642
Entry No 643
Entry No 644
Entry No 645
Entry No 646
Entry No 647
Entry No 648
Entry No 649
Entry No 650
Entry No 651
Entry No 652
Entry No 653
Entry No 654
Entry No 655
Entry No 656
Entry No 657
Entry No 658
Entry No 659
Entry No 660
Entry No 661
Entry No 662
Entry No 663
Entry No 664
Entry No 665
Entry No 666
Entry No 667
Entry No 668
Entry No 669
Entry No 670
Entry No 671
Entry No 672
Entry No 673
Entry No 674
Entry No 675
Entry No 676
Entry No 677
Entry No 678
Entry No 679
Entry No 680
Entry No 681
Entry No 682
Entry No 683
Entry No 684
Entry No 685
Entry No 686
Entry No 687
Entry No 688
Entry No 689
Entry No 690
Entry No 691
Entry No 692
Entry No 693
Entry No 694
Entry No 695
Entry No 696
Entry No 697
Entry No 698
Entry No 699
Entry No 700
Entry No 701
Entry No 702
Entry No 703
Entry No 704
Entry No 705
Entry No 706
Entry No 707
Entry No 708
Entry No 709
Entry No 710
Entry No 711
Entry No 712
Entry No 713
Entry No 714
Entry No 715
Entry No 716

Entry No 1251
Entry No 1252
Entry No 1253
Entry No 1254
Entry No 1255
Entry No 1256
Entry No 1257
Entry No 1258
Entry No 1259
Entry No 1260
Entry No 1261
Entry No 1262
Entry No 1263
Entry No 1264
Entry No 1265
Entry No 1266
Entry No 1267
Entry No 1268
Entry No 1269
Entry No 1270
Entry No 1271
Entry No 1272
Entry No 1273
Entry No 1274
Entry No 1275
Entry No 1276
Entry No 1277
Entry No 1278
Entry No 1279
Entry No 1280
Entry No 1281
Entry No 1282
Entry No 1283
Entry No 1284
Entry No 1285
Entry No 1286
Entry No 1287
Entry No 1288
Entry No 1289
Entry No 1290
Entry No 1291
Entry No 1292
Entry No 1293
Entry No 1294
Entry No 1295
Entry No 1296
Entry No 1297
Entry No 1298
Entry No 1299
Entry No 1300
Entry No 1301
Entry No 1302
Entry No 1303
Entry No 1304
Entry No 1305
Entry No 1306
Entry No 1307
Entry No 1308
Entry No 1309
Entry No 1310
Entry No 1311
Entry No 1312
Entry No 1313
Entry No 1314
Entry No 1315
Entry No 1316
Entry No 1317
Entry No 1318
Entry No 1319
Entry No 1320
Entry No 1321
Entry 